In [1]:
import pandas as pd

In [2]:
def getUnitPreferences():
    unitTemp = ''
    while ((unitTemp != 'F') and (unitTemp != 'C')):
        print("-----------------------------------------")
        print("Would you like to enter your temperature in\n"
            "Fahrenheit (F) or Celcius (C). Enter F or C:", end=' ')
        unitTemp = input()
        if ((unitTemp != 'F') and (unitTemp != 'C')):
            print('\nInvalid Input')
        elif (unitTemp == 'F'):
            unitTempFull = 'Fahrenheit'     
        else:
            unitTempFull = 'Celcius'

    unitSnow = ''
    while ((unitSnow != 'cm') and (unitSnow != 'in')):            
        print("\n-----------------------------------------")
        print("Would you like to enter your snowfall in\n"
            "Centimeters (cm) or Inches (in). Enter cm or in:", end=' ')
        unitSnow = input()
        if ((unitSnow != 'cm') and (unitSnow != 'in')):
            print('\nInvalid Input')
        elif (unitSnow == 'cm'):
            unitSnowFull = 'Centimeters'     
        else:
            unitSnowFull = 'Inches'

    unitRain = ''
    while ((unitRain != 'cm') and (unitRain != 'in')):            
        print("\n-----------------------------------------")
        print("Would you like to enter your rainfall in\n"
            "Centimeters (cm) or Inches (in). Enter cm or in:", end=' ')
        unitRain = input()
        if ((unitRain != 'cm') and (unitRain != 'in')):
            print('\nInvalid Input')
        elif (unitRain == 'cm'):
            unitRainFull = 'Centimeters'     
        else:
            unitRainFull = 'Inches'

    return [unitTempFull, unitSnowFull, unitRainFull]

In [3]:
def getWeatherPreferences():
      print("-----------------------------------------")
      print("Welcome to the European Livability Index!\n"
            "The goal of this program is to take in your\n"
            "living preferences and provide you with a\n"
            "list of places you might enjoy!\n\n"
            "Currently the program creates your livability\n"
            "matching score based on desired temperature,\n"
            "rainfall, and snowfall for every season")

      unitChoices = getUnitPreferences()  

      print("\n-----------------------------------------")
      print("Enter your desired temperature in " + unitChoices[0] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      temperature = input().split(" ")
      print("-----------------------------------------")
      print("Enter your desired snowfall in " + unitChoices[1] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      snow = input().split(" ")
      print("-----------------------------------------")
      print("Enter your desired rainfall in " + unitChoices[2] + " for every\n"
            "season in this format Spring Summer Fall Winter")
      print("-----------------------------------------")
      print("Enter choices here:")
      rain = input().split(" ")
      print("-----------------------------------------")

      return [temperature, snow, rain], unitChoices


In [17]:
def normalizeDifference(actual, preference, sIndex):
    difference = abs(actual - float(preference[sIndex]))
    min = difference.min()
    max = difference.max()
    differenceNormalized = (difference - min) / (max - min)
    return differenceNormalized

In [18]:
def computerWeatherScore(choices, unitChoices):

    weatherData = pd.read_csv("../Data/Weather/weatherData.csv")
    springWeather = weatherData[weatherData['SEASON'] == 'Spring']
    summerWeather = weatherData[weatherData['SEASON'] == 'Summer']
    fallWeather = weatherData[weatherData['SEASON'] == 'Fall']
    winterWeather = weatherData[weatherData['SEASON'] == 'Winter']
    weatherDataSeasons = [springWeather, summerWeather, fallWeather, winterWeather]

    if (unitChoices[0] == 'Fahrenheit'):
        for index in range(len(choices[0])):
            choices[0][index] = (float(choices[0][index]) - 32.0) * (5/9)
    if (unitChoices[1] == 'Inches'):
        for index in range(len(choices[1])):
            choices[1][index] = (float(choices[1][index]) * 2.54)
        choices[1] = choices[1]
    if (unitChoices[2] == 'Inches'):
        for index in range(len(choices[2])):
            choices[2][index] = (float(choices[2][index]) * 2.54)

    scoreData = pd.DataFrame()
    sIndex = 0

    for season in weatherDataSeasons:
        season['SCORE'] = normalizeDifference(season['Average Temp (C)'], choices[0], sIndex) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)  
        scoreData = pd.concat([scoreData, season])
        sIndex+=1
   
    return scoreData

    

In [10]:
choices, unitChoices = getWeatherPreferences()

-----------------------------------------
Welcome to the European Livability Index!
The goal of this program is to take in your
living preferences and provide you with a
list of places you might enjoy!

Currently the program creates your livability
matching score based on desired temperature,
rainfall, and snowfall for every season
-----------------------------------------
Would you like to enter your temperature in
Fahrenheit (F) or Celcius (C). Enter F or C: 
Invalid Input
-----------------------------------------
Would you like to enter your temperature in
Fahrenheit (F) or Celcius (C). Enter F or C: 
Invalid Input
-----------------------------------------
Would you like to enter your temperature in
Fahrenheit (F) or Celcius (C). Enter F or C: 
Invalid Input
-----------------------------------------
Would you like to enter your temperature in
Fahrenheit (F) or Celcius (C). Enter F or C: 
-----------------------------------------
Would you like to enter your snowfall in
Centimeters (

In [19]:
scores = computerWeatherScore(choices, unitChoices)
scoresDropped = scores.drop(columns=['SEASON', 'Average Temp (C)', 'Average Snow Depth (cm)', 'Average Rainfall (cm)'])
scoresRegion = scoresDropped.groupby(['MUNICIPALITY', 'COUNTRY']).agg({'SCORE': 'mean'})
scoresRegion = scoresRegion.sort_values(by=['SCORE'])

C:\Users\austi\AppData\Local\Temp\ipykernel_9592\377945527.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season['SCORE'] = normalizeDifference(season['Average Temp (C)'], choices[0], sIndex) + normalizeDifference(season['Average Snow Depth (cm)'], choices[1], sIndex) + normalizeDifference(season['Average Rainfall (cm)'], choices[2], sIndex)
C:\Users\austi\AppData\Local\Temp\ipykernel_9592\377945527.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season['SCORE'] = normalizeDifference(season['Aver

In [20]:
scoresRegion.iloc[:10]

,,SCORE
MUNICIPALITY,COUNTRY,
Appenzell Innerrhoden,Switzerland,0.703597
Radovljica,Slovenia,0.956546
Idrija,Slovenia,1.029170
Kujalleq Municipality,Greenland,1.111286
Kolašin Municipality,Montenegro,1.138554
Vestland,Norway,1.168077
Sermersooq Municipality,Greenland,1.301392
Lika-Senj County,Croatia,1.320430
Žabljak Municipality,Montenegro,1.329048


In [11]:
weatherData = pd.read_csv("../Data/Weather/weatherData.csv")
weatherData[weatherData['COUNTRY'] == 'United Kingdom']

,SEASON,MUNICIPALITY,COUNTRY,Average Temp (C),Average Snow Depth (cm),Average Rainfall (cm)
49,Fall,England,United Kingdom,10.077748,0.034600,2.783308
127,Fall,Northern Ireland,United Kingdom,9.237396,0.034171,3.730784
180,Fall,Scotland,United Kingdom,8.192950,0.149547,3.815249
229,Fall,Wales,United Kingdom,10.637580,0.002336,3.232054
295,Spring,England,United Kingdom,9.030614,0.061302,1.865519
373,Spring,Northern Ireland,United Kingdom,8.416274,0.020141,2.504044
426,Spring,Scotland,United Kingdom,7.508692,0.093538,2.392142
475,Spring,Wales,United Kingdom,9.151167,0.000000,1.787470
540,Summer,England,United Kingdom,16.058219,0.001236,2.233789
618,Summer,Northern Ireland,United Kingdom,14.465985,0.000000,3.064157
